In [ ]:
# default_exp model

In [ ]:
# hide
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import pandas as pd 
import numpy as np
from nbdev.showdoc import show_doc

# model 

> help library

In [ ]:
# export 
class ForwardHook():
    def __init__(self, module, name:None, activation:bool, stats:bool):
        ''' Track stats and activations of any layer in model '''
        self.hook = module.register_forward_hook(self.hook_fn)
        self.status = 'active'
        self.name = name
        self.activation_status = activation
        self.stats = stats
        if self.activation_status: self.activations = []
        if stats: self.means, self.stds = [], []

    def hook_fn(self, module, input, output):
        if self.activation_status: self.activations.append ( output[1].detach().cpu().numpy() )
        if self.stats: 
          self.means += output[1].mean(1).detach().cpu().numpy().squeeze().tolist() 
          self.stds += output[1].std(1).detach().cpu().numpy().squeeze().tolist()

    def _stack_activations(self):
        self.activations = np.vstack(self.activations)

    def close(self):
        self.hook.remove()
        self.status = 'removed'

Inputs: <br>
- module(nn.Module): Module example: model.linear1
- name(str): Tag 
- activation(bool): if True, it will track module output
- stats(bool): if True, it will track module output mean and std 

## `_stack_activations`

This class can be inheritet this class and define your own `hook_fn`

In [ ]:
#export
def get_linear_layer_activations_states(dls, model, layers:list, stats:bool, activation:bool, remove_hooks=True):

  hooks = dict()
  output_dict = {'senti': list(), 'lang': list()}

  for layer in layers:
    if hasattr(model, layer): hooks[f'{layer}'] = ForwardHook ( getattr(model, layer),layer, activation=activation, stats=stats ) 

  model.eval()
  i = 0
  for batch in tqdm(dls, desc='Fatching data: '):
    with torch.no_grad():
        i+=1
        pred_senti, pred_lang = model(batch['ids'], batch['mask'], batch['token_type_ids'])
        output_dict['senti'] += pred_senti.sigmoid().round().squeeze().detach().cpu().numpy().tolist()
        output_dict['lang'] += pred_lang.sigmoid().round().squeeze().detach().cpu().numpy().tolist()

  if stats: means, stds = dict(), dict()
  if activation: activations = dict()

  for k,v in hooks.items():
    if activation: v._stack_activations()
    if remove_hooks: v.close()

  return hooks, output_dict